In [1]:
#upload file
from google.colab import files
uploaded = files.upload()

Saving Final-Sample-External-1.xlsx - Final Raw Sample(0%).csv to Final-Sample-External-1.xlsx - Final Raw Sample(0%).csv


Import some useful libiaries

In [2]:
# import libiaries:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Read the file in dataframe and get some insights of the data

In [10]:
# Read data
df = pd.read_csv('/content/Final-Sample-External-1.xlsx - Final Raw Sample(0%).csv')
df.head()

,Year,Company Name,Country,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity,SDG 1.5,SDG 2.1,SDG 2.2,SDG 2.3,SDG 2.4,SDG 3.3,SDG 3.4,SDG 3.9,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
0,2019,Saga plc,United Kingdom,Activities auxiliary to financial intermediati...,-2.89%,-13.03%,"(31,842,309)","(31,150,754)","(7,184)","(414,256)","(97,118)","(1,302)","(1,059)","(170,776)",141,"(13,942,797)","(8,142,779)","(8,139,160)","(103,564)","(103,564)","(1,312,787)","79,125",(196),"(170,776)","(1,059)",(5),(1),"(3,585)",(6),71,71,"(1,297)",1%
1,2019,BURSA MALAYSIA BHD,Malaysia,Activities auxiliary to financial intermediati...,-1.68%,-3.47%,"(1,968,379)","(1,924,910)",(451),"(25,349)","(5,938)",(81),(168),"(11,502)",20,"(852,646)","(502,708)","(502,460)","(6,337)","(6,337)","(81,118)","(4,791)",(27),"(11,502)",(168),(1),(1),(222),(2),10,10,(79),4%
2,2019,INTERTEK GROUP PLC,United Kingdom,Activities auxiliary to financial intermediati...,-1.53%,-9.49%,"(60,599,272)","(59,281,663)","(13,774)","(788,289)","(184,802)","(2,487)","(3,804)","(324,960)",508,"(26,533,166)","(15,557,810)","(15,550,827)","(197,072)","(197,072)","(2,509,207)","284,215",(703),"(324,960)","(3,804)",(17),(4),"(6,861)",(20),254,254,"(2,470)",1%
3,2019,JSE LIMITED,South Africa,Activities auxiliary to financial intermediati...,-1.46%,NaN,"(2,290,124)","(2,239,814)",(510),"(29,662)","(6,938)",(93),(901),"(12,200)",(6),"(995,881)","(576,811)","(576,488)","(7,415)","(7,415)","(92,910)","(19,470)",(277),"(12,200)",(901),(0),(1),(253),(0),(3),(3),(93),2%
4,2019,BUREAU VERITAS SA,France,Activities auxiliary to financial intermediati...,-0.70%,-5.10%,"(39,978,650)","(39,107,612)","(9,330)","(520,701)","(121,953)","(1,671)","(4,116)","(214,438)","1,172","(17,514,837)","(10,430,409)","(10,425,281)","(130,175)","(130,175)","(1,684,676)","561,195",(577),"(214,438)","(4,116)",(38),(9),"(4,607)",(45),586,586,"(1,633)",3%


Clean out the column names and format columns from the csv to float.

In [11]:
column_list = []
for column in df.columns:
    column_list.append(column.replace(' ', ''))
df.columns = column_list
df = df[['Year', 'Country', 'Industry(Exiobase)', 'EnvironmentalIntensity(Sales)', 'TotalEnvironmentalCost']]

def percent_to_float(s):
    return float(s.strip('%')) / 100.0

df['EnvironmentalIntensity(Sales)'] = df['EnvironmentalIntensity(Sales)'].apply(percent_to_float)

replace_dict = {'(':'',')':'', ' ' : '', ',' : ''}
def paranthesis_to_minus(value):
    for i, j in replace_dict.items():
        value = value.replace(i, j)
    value = int(f'-{value}')
    return value

df['TotalEnvironmentalCost'] = df['TotalEnvironmentalCost'].apply(paranthesis_to_minus)


Country:

In [12]:
#create dummies for country:
df = pd.get_dummies(df, columns = ['Country'])

x = df.drop(columns = ['Year',  'EnvironmentalIntensity(Sales)', 'TotalEnvironmentalCost','Industry(Exiobase)'])
y = df['EnvironmentalIntensity(Sales)']

In [13]:
#Setup the dataframe to fit into the models. Fit the models.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
OLS_VS = sm.OLS(y_train, x_train).fit()
y_pred = OLS_VS.predict(x_test)
r2_score(y_test, y_pred)

0.06869355423668577

Create models and see the R^2

In [10]:
OLS_CV = linear_model.LinearRegression()
lasso = linear_model.Lasso()
ridge = linear_model.Ridge()

OLS_CV.fit(x_train, y_train)
y_pred = OLS_CV.predict(x_test)
r2_score(y_test, y_pred)

print(cross_val_score(OLS_CV, x_train, y_train, cv=3, scoring = 'r2'))
print(cross_val_score(lasso, x_train, y_train, cv=3, scoring = 'r2'))
print(cross_val_score(ridge, x_train, y_train, cv=3, scoring = 'r2'))

[-1.58147403e+23 -1.13698965e+24 -1.10982061e+22]
[-0.00019949 -0.00172387 -0.00051852]
[0.04458907 0.02144513 0.05797352]


In [ ]:
#Try more on  Ridge
ridge2 = Ridge(alpha = 4, normalize = True)
ridge2.fit(x_train, y_train)             # Fit a ridge regression on the training data
pred2 = ridge2.predict(x_test)           # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = x.columns)) # Print coefficients
print(mean_squared_error(y_test, pred2))          # Calculate the test MSE when alpha=4

Country_ Argentina              0.019551
Country_ Australia             -0.005811
Country_ Austria                0.001120
Country_ Belgium                0.008106
Country_ Bermuda                0.110859
                                  ...   
Country_Turkey                 -0.096032
Country_Ukraine                -0.009798
Country_United Arab Emirates    0.037884
Country_United Kingdom          0.012517
Country_United States          -0.010405
Length: 129, dtype: float64
0.06970233041366569


In [14]:
#MSE decreases when alpha=0
ridge2 = Ridge(alpha = 0, normalize = True)
ridge2.fit(x_train, y_train)             # Fit a ridge regression on the training data
pred = ridge2.predict(x_test)            # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = x.columns)) # Print coefficients
print(mean_squared_error(y_test, pred))           # Calculate the test MSE
print(r2_score(y_test, pred))

Country_ Argentina             -1.224630e+12
Country_ Australia             -1.224630e+12
Country_ Austria               -1.224630e+12
Country_ Belgium               -1.224630e+12
Country_ Bermuda               -1.224630e+12
                                    ...     
Country_Turkey                 -1.224630e+12
Country_Ukraine                -1.224630e+12
Country_United Arab Emirates   -1.224630e+12
Country_United Kingdom         -1.224630e+12
Country_United States          -1.224630e+12
Length: 129, dtype: float64
5.16609844370155e+20
-7.23691527615599e+21


In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(x_train, y_train)
ridgecv.alpha_
ridge4 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge4.fit(x_train, y_train)
mean_squared_error(y_test, ridge4.predict(x_test))

0.06668199359489732

In [ ]:
# Try more on Lasso:
lassoo = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lassoo.set_params(alpha=a)
    lassoo.fit(scale(x_train), y_train)
    coefs.append(lassoo.coef_)
lassocv = LassoCV(alphas = None, cv = 2, max_iter = 100000, normalize = True)
lassocv.fit(x_train, y_train)

lassoo.set_params(alpha=lassocv.alpha_)
lassoo.fit(x_train, y_train)
mean_squared_error(y_test, lassoo.predict(x_test))

0.06705130667221469

In [ ]:
pd.Series(lassoo.coef_, index=x.columns)

Country_ Argentina              0.000000
Country_ Australia             -0.015843
Country_ Austria                0.000000
Country_ Belgium                0.000000
Country_ Bermuda                0.441436
                                  ...   
Country_Turkey                 -0.430045
Country_Ukraine                -0.000000
Country_United Arab Emirates    0.066342
Country_United Kingdom          0.045519
Country_United States          -0.043993
Length: 129, dtype: float64

In [14]:
#Random Forest
X, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)

y_pred = regr.predict(x_test)
print('Random Forest:', r2_score(y_test, y_pred))

Random Forest: 0.008440791625494382
